In [ ]:
import numpy as np
import sys
import cv2
from PIL import Image
import math
from scipy import signal
import time

In [ ]:
left = cv2.imread('HW2/tsukuba1.ppm',0)
right = cv2.imread('HW2/tsukuba2.ppm',0)
h,w = left.shape
depth_ssd = np.zeros((h,w),np.uint8)
depth_ncc = np.zeros((h,w),np.uint8)

In [ ]:
#window size - n
n = 5
w_size = int(n/2)
#pixel search range - psr
psr = 30
adjust_op = 255/psr

In [ ]:
#ssd - sum squared distance
start = time.time()
for j in range(w_size, h - w_size):
    for i in range(w_size, w - w_size):
        ssd_depth_val = 0
        prev_ssd = sys.maxsize
        #moving along scanline
        for d in range(psr):                                         
            M1 = left[j-w_size:j+w_size,i-w_size:i+w_size]
            M2 = right[j-w_size:j+w_size,i-w_size-d:i+w_size-d]
            #calculating ssd
            if(M2.shape != M1.shape):
                M2=np.zeros(M1.shape)
            M=M1-M2
            M=M**2
            ssd = np.sum(M)
            #finding window that gives min. ssd
            if ssd < prev_ssd:
                prev_ssd = ssd
                ssd_depth_val = d
        depth_ssd[j, i] = ssd_depth_val * adjust_op
Image.fromarray(depth_ssd).save('depth_ssd.png')
end = time.time()
print("Time taken by ssd :",(end-start))

In [ ]:
#ncc - normalized cross-correlation
start = time.time()
for j in range(w_size, h - w_size):
    for i in range(w_size, w - w_size):
        ncc_disp_val = 0
        prev_ncc = -sys.maxsize -1
        #moving along scanline
        for d in range(psr):
            tncc = 0
            temp_ncc = 0
            M1 = left[j-w_size:j+w_size,i-w_size:i+w_size]
            M2 = right[j-w_size:j+w_size,i-w_size-d:i+w_size-d]
            mean1 = np.mean(M1)
            mean2 = np.mean(M2)
            std1 = np.std(M1)
            std2 = np.std(M2)
            #calculating ncc
            for v in range(-w_size, w_size):
                for u in range(-w_size, w_size):
                    temp_ncc = (left[j+v, i+u]-mean1)*(right[j+v, i+u - d]-mean2)
                    temp_ncc /= (std1*std2)
                    tncc += temp_ncc
            tncc /= n**2
            #finding window that gives max. ncc
            if tncc>prev_ncc:
                prev_ncc = tncc
                ncc_disp_val = d
        depth_ncc[j, i] = ncc_disp_val * adjust_op
end = time.time()
print(end-start)
print("Time taken by normalised cross-correlation : "+str(end-start))
Image.fromarray(depth_ncc).save('ncc_w_7_psr_30.jpg')